<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
#default_exp trainer.base

In [ ]:
#export

class TTSTrainer():

    def __init__(model_type, model_opts, data, warm_start = False):

        self.data = data

        if not warm_start:
            pass
            # model = ud.TTSModel.create(model_type, model_options)


    def train():
        pass
        #for batch in enumerate(data):
        #    #fill in